In [ ]:
import os
import random
from datetime import datetime
import ast
import math

import gc
import yaml
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter

from preprocessing.final_preprocessing import serve_data_unet

from models.context_unet import ContextUnet

import ddpm.cu_ddpm as ddpm

from evaluation.pca_tsne import visualize_pca_tsne_unet
from evaluation.jsd import compute_jsd
from evaluation.rmse import rmse

gc.collect()

random.seed(42)
np.random.seed(42)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
config_path = "./config_unet.yml"
with open(config_path, 'r') as f:
    config = yaml.full_load(f)

date = datetime.today().strftime('%Y-%m-%d-%H-%M')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

epochs = config["epochs"]
timesteps = config["timesteps"]
batch_size_config = config["batch_size"]
latent_dim = config["latent_dim"]
num_layers = config["num_layers"]
n_heads = config["n_heads"]
beta_schedule = config["beta_schedule"]
objective = config["objective"]
model_name = config["model_name"]
cond_model = config["cond_model"]
lr = float(config["lr"])
betas = tuple_of_floats = ast.literal_eval(config["betas"])
save_rate = int(config["save_rate"])
img_rows = int(config["img_rows"])
img_cols = int(config["img_cols"])
channels = int(config["channels"])
n_feat = int(config["n_feat"])

assert model_name in ["ContextUnet"], "Chosen  model was not valid, the option is ContextUnet"
assert cond_model in {"mlp", "te", "stft"}, "Chosen conditioning model was not valid, the options are mlp, te and stft"


log_file_name = f"{model_name}_{cond_model}_{str(date)}"
tb_writer = f"./logging/tensorboard/{log_file_name}"

# Data

In [ ]:
#Can take a while to load
train_loader, test_loader, n_cfeat, customer_ids, train_data, test_data = serve_data_unet(batch_size=batch_size_config)

# Model

In [ ]:
model = ContextUnet(in_channels=channels, n_feat=n_feat, n_cfeat=n_cfeat, height=img_cols, cond_model=cond_model).to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)
writer = SummaryWriter(tb_writer)

## DDPM

# Train

In [ ]:
run_path = f"./logging/weights/{log_file_name}"
os.makedirs(run_path)

rmse_list = []
lowest_loss = 100000
step=0

model.train()
for ep in tqdm(range(epochs)):
    # linearly decay learning rate
    optim.param_groups[0]['lr'] = lr*(1-ep/epochs)
    epoch_loss = 0.0   # initialize epoch loss
    for i, (x, c) in enumerate(train_loader):   # x: images  c: context
        optim.zero_grad()
        x = x.to(device)
        c = c.to(x)
        
        # randomly mask out c
        context_mask = torch.bernoulli(torch.zeros(c.shape[0]) + 0.9).to(device)
        c = c * context_mask.unsqueeze(-1)

        # perturb data
        noise = torch.randn_like(x)
        noise = noise.type(torch.float32)
        t = torch.randint(1, timesteps + 1, (x.shape[0],)).to(device)
        x_pert = ddpm.perturb_input(x, t, noise, beta1=betas[0], beta2=betas[1], timesteps=timesteps, device=device)

        # use network to recover noise
        time = t / timesteps
        time = time.type(torch.float32)
        x_pert = x_pert.type(torch.float32)
        c = c.type(torch.float32)
        
        pred_noise = model(x_pert, time, c=None)
        # loss is mean squared error between the predicted and true noise
        loss = F.mse_loss(pred_noise, noise)
        epoch_loss += loss.item()
        loss.backward()
        optim.step()
        
        
        if loss.item() < lowest_loss:
            lowest_loss = loss.item()
            best_model_params = model.state_dict()
        
        writer.add_scalar("Batch Training Loss", loss, global_step=step)
        step += 1
    
    avg_epoch_loss = epoch_loss / len(train_loader)
    writer.add_scalar("Epoch Training Loss", avg_epoch_loss, ep)
    
    if ep % 5 ==0:        
        sample, _ = ddpm.sample_ddpm_context(n_sample=10, context=None, channels=channels, height=img_cols, 
                                      device=device, timesteps=timesteps, nn_model=model, beta1=betas[0], beta2=betas[1])
        
        p = sample.repeat(1, 3, 1, 1).cpu()
        q = torch.from_numpy(test_data[:batch_size_config]).repeat(1, 3, 1, 1) 
        
        rmse_res = rmse(p,q).item()
        rmse_list.append(rmse_res)   
        
        writer.add_scalar('Epoch Training RMSE', rmse_res, ep)
        
        print(f'Epoch: {ep+1}, Epoch Loss: {avg_epoch_loss:.4f}, RMSE: {rmse(p,q):.4f}')
    
    else:
        print(f'Epoch: {ep+1}, Epoch Loss: {avg_epoch_loss:.4f}')
        
    if ep % save_rate == 0:
        torch.save({
            'epoch': ep+1,
            'diffusion_state_dict': model.state_dict(),
            'diffusion_optim_state_dict': optim.state_dict()
            }, os.path.join(f'{run_path}', f'ep_{ep}_mse_{avg_epoch_loss:.3f}_weights.pth'))
    
torch.save(best_model_params, os.path.join(f'{run_path}', 'best_model_weights.pth'))

# Sampling

In [ ]:
def plot_samples(sample, rows=None, cols=None, figsize=(12, 10), title=None):
    n_images = sample.shape[0]
    
    if rows is None and cols is None:
        cols = math.ceil(math.sqrt(n_images))
        rows = math.ceil(n_images / cols)
    elif rows is None:
        rows = math.ceil(n_images / cols)
    elif cols is None:
        cols = math.ceil(n_images / rows)
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    
    if title:
        fig.suptitle(title, fontsize=16)
    
    if rows > 1 or cols > 1:
        axes = axes.flatten()
    else:
        axes = [axes]  
    
    for i in range(n_images):
        if i < len(axes):  
            im = axes[i].imshow(sample[i], cmap='viridis')
            axes[i].set_title(f'Image {i+1}')
            axes[i].axis('off')  
    
    for i in range(n_images, len(axes)):
        axes[i].axis('off')
        
    plt.tight_layout()
    plt.show()

In [ ]:
sample, _ = ddpm.sample_ddpm_context(n_sample=10, context=None, channels=channels, height=img_cols, 
                                      device=device, timesteps=timesteps, nn_model=model, beta1=betas[0], beta2=betas[1])

In [ ]:
sample = sample.cpu().numpy()
sample = sample.squeeze()

In [ ]:
plot_samples(sample, title="No conditioning train")

# Evaluation
## Without Conditioning
### Train

In [ ]:
real_data_train, real_cond_data_train = next(iter(train_loader))
real_data_test, real_cond_data_test = next(iter(test_loader))

real_cond_data_train.shape, real_data_test.shape

In [ ]:
selected_customer_ids = random.sample(range(len(customer_ids)), 5)

In [ ]:
visualize_pca_tsne_unet(real_data_train, sample, customer_ids, filename=log_file_name)

In [ ]:
jsd_train_no_cond = {}
multi_customer_sample = sample.reshape(-1, img_cols*img_rows)
multi_customer_real_data = real_data_train.reshape(-1, img_cols*img_rows)

for i in range(multi_customer_sample.shape[1]):
    jsd_train_no_cond[i] = compute_jsd(multi_customer_real_data[:, i], multi_customer_sample[:, i])

### Test

In [ ]:
visualize_pca_tsne_unet(real_data_test, sample, customer_ids, filename=log_file_name, train_test="Test")

In [ ]:
jsd_test_no_cond = {}
multi_customer_sample = sample.reshape(-1, img_cols*img_rows)
multi_customer_real_data = real_data_test.reshape(-1, img_cols*img_rows)

for i in range(multi_customer_sample.shape[1]):
    jsd_test_no_cond[i] = compute_jsd(multi_customer_real_data[:, i], multi_customer_sample[:, i])

## With Conditioning
### Train

In [ ]:
sample, _ = ddpm.sample_ddpm_context(n_sample=10, context=real_cond_data_train.to(device), channels=channels, height=img_cols, 
                                      device=device, timesteps=timesteps, nn_model=model, beta1=betas[0], beta2=betas[1])
sample = sample.cpu().numpy()
sample = sample.squeeze()

In [ ]:
visualize_pca_tsne_unet(real_data_train, sample, customer_ids, filename=log_file_name, cond=True)

In [ ]:
jsd_train_cond = {}
multi_customer_sample = sample.reshape(-1, img_cols*img_rows)
multi_customer_real_data = real_data_train.reshape(-1, img_cols*img_rows)

for i in range(multi_customer_sample.shape[1]):
    jsd_train_cond[i] = compute_jsd(multi_customer_real_data[:, i], multi_customer_sample[:, i])

### Test

In [ ]:
sample, _ = ddpm.sample_ddpm_context(n_sample=10, context=real_cond_data_test.to(device), channels=channels, height=img_cols, 
                                      device=device, timesteps=timesteps, nn_model=model, beta1=betas[0], beta2=betas[1])
sample = sample.cpu().numpy()
sample = sample.squeeze()

In [ ]:
visualize_pca_tsne_unet(real_data_test, sample, customer_ids, filename=log_file_name, cond=True, train_test="Test")

In [ ]:
jsd_test_cond = {}
multi_customer_sample = sample.reshape(-1, img_cols*img_rows)
multi_customer_real_data = real_data_test.reshape(-1, img_cols*img_rows)

for i in range(multi_customer_sample.shape[1]):
    jsd_test_cond[i] = compute_jsd(multi_customer_real_data[:, i], multi_customer_sample[:, i])

In [ ]:
jsd_data = pd.DataFrame({
    'Train No Cond': jsd_train_no_cond,
    'Test No Cond': jsd_test_no_cond,
    'Train Cond': jsd_train_cond,
    'Test Cond': jsd_test_cond
})

all_keys = set(jsd_train_no_cond.keys()) | set(jsd_test_no_cond.keys()) | set(jsd_train_cond.keys()) | set(jsd_test_cond.keys())

jsd_data = pd.DataFrame({
    'Train No Cond': [jsd_train_no_cond.get(k, None) for k in all_keys],
    'Test No Cond': [jsd_test_no_cond.get(k, None) for k in all_keys],
    'Train Cond': [jsd_train_cond.get(k, None) for k in all_keys],
    'Test Cond': [jsd_test_cond.get(k, None) for k in all_keys]
}, index=list(all_keys))

In [ ]:
plot_dir = f'./logging/plots/JSD/'

if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)


means = jsd_data.mean()

plt.figure(figsize=(10, 6))
bars = plt.bar(means.index, means.values)

plt.title('Average JSD Values for all Customers', fontsize=15)
plt.ylabel('Average Value', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(os.path.join(plot_dir, log_file_name+".png", ))
plt.show()

In [ ]:
# plot_dir = f'./logging/plots/JSD/'

# if not os.path.exists(plot_dir):
#         os.makedirs(plot_dir)

# fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
# axs = axs.flatten()

# for i, column in enumerate(jsd_data.columns):
#     axs[i].bar(jsd_data.index, jsd_data[column])
#     axs[i].set_title(column)
#     axs[i].set_xlabel('Customers')
#     axs[i].set_ylabel('JSD')
#     axs[i].tick_params(axis='x', rotation=45)

# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, log_file_name+".png", ))
# plt.show()

In [ ]:
log_dir = "./logging/logs"
log_file_path = os.path.join(log_dir, log_file_name + ".txt")

jsd_data.to_csv(os.path.join(log_dir, log_file_name + ".csv"))

with open(log_file_path, 'w') as log_file:
    log_file.write("Config:\n")
    for key, value in config.items():
        log_file.write(f"{key}: {value}\n")
    
    log_file.write("\n\nRMSE Values:\n")
    for rmse in rmse_list:
        log_file.write(f"{rmse}\n")
    
    log_file.write("\n\nJSD Values:\n")
    log_file.write(str(os.path.join(log_dir, log_file_name + ".csv")))

print(f"Log file created at: {log_file_path}")